<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/HuggingFace_int8_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace meets `bitsandbytes` for lighter models on GPU for inference

 <center>
 <img src="https://s3.amazonaws.com/moonup/production/uploads/1659861207959-62441d1d9fdefb55a0b7d12c.png">
 </center>


You can run your own 8-bit model on any HuggingFace 🤗 model with just few lines of code. Install the dependencies below first!


In [1]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Hardware requirements 🔨

To run properly this feature you need to have GPU that supports 8-bit operation modules. Currently, Turing and Ampere GPUs (RTX20s, RTX30s, A40-A100, T4+) are supported, which means on colab we need to use a T4 GPU for this feature. You can check that using this code snippet and make sure you are using a supported GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Feb  4 07:07:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Here we are using a `Tesla T4` GPU that should support 8-bit tensor cores! We are good to go 🚀

## Utility variables & functions 🧰

In [5]:
name = "bigscience/bloom-3b"
max_new_tokens = 20

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

## Use 8bit models and `pipeline` 🤗

You can use 8bit quantized models together with `pipeline` as follows:

In [7]:
from transformers import pipeline

pipe = pipeline(model=name, model_kwargs= {"device_map": "auto", "load_in_8bit": True}, max_new_tokens=max_new_tokens)

/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/paths.py:110: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(


CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA path found: /usr/local/cuda/lib64/libcudart.so
CUDA_SETUP: Detected CUDA version 112
CUDA_SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


Let's check the output!

In [14]:
text = "this is a poem about apple\n\nAn apple a day is"
pipe(text, max_new_tokens =100)

[{'generated_text': 'this is a poem about apple\n\nAn apple a day is a life long friend\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps the doctor away\n\nAn apple a day keeps'}]

## Use 8bit models and `.generate` 📖

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit = AutoModelForCausalLM.from_pretrained("bigscience/bloom-7b1", device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/paths.py:110: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(


CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA path found: /usr/local/cuda/lib64/libcudart.so
CUDA_SETUP: Detected CUDA version 112
CUDA_SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text = "this is a poem about apple\n\nAn apple a day is"
generate_from_model(model_8bit, tokenizer)

Let's compare the qualitative results between our quantized model and the original model

In [17]:
model_native = AutoModelForCausalLM.from_pretrained(name, device_map="auto", torch_dtype="auto")
generate_from_model(model_native, tokenizer)

'this is a poem about apple\n\nAn apple a day is a life long friend\n\nAn apple a'

## Memory footprint comparison 🪶

In [ ]:
mem_fp16 = model_native.get_memory_footprint()
mem_int8 = model_8bit.get_memory_footprint()
print("Memory footprint int8 model: {} | Memory footprint fp16 model: {} | Relative difference: {}".format(mem_int8, mem_fp16, mem_fp16/mem_int8))

Memory footprint int8 model: 3645818880 | Memory footprint fp16 model: 6005114880 | Relative difference: 1.6471237539918604


We saved 1.65x memory for a 3-billion parameters models! Note that internally we replace all the linear layers by the ones implemented in `bitsandbytes`. By scaling up the model the number of linear layers will increase therefore the impact of saving memory on those layers will be huge for very large models. For example quantizing BLOOM-176 (176 Billion parameter model) gives a gain of 1.96x memory footprint which can save a lot of compute power in practice.

## Hyper-parameter tuning 📠


**Warning:** you may want to run these cells separately from previous cells to avoid Out Of Memory (OOM) issues.

You can play with the parameter `int8_threshold` and see its impact in the results of your model. You can directly specify this parameter when loading your model through `.from_pretrained` method. By default we set this parameter to be `6.0` as described in the paper.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit_thresh_4 = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=4.0)
model_8bit_thresh_2 = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=2.0)
tokenizer = AutoTokenizer.from_pretrained(name)

In [ ]:
generate_from_model(model_8bit_thresh_4, tokenizer)

'Hello my name is John and I am a student at the University of the West Indies. I am'

In [ ]:
generate_from_model(model_8bit_thresh_2, tokenizer)

'Hello my name is John and I am a newbie to the forum. I have a question about'

As you can see the generations can slightly vary by using different thresholds. This is because manipulating 8-bit parameters leads to easier perturbations by small changes! Lowering the threshold means also less parameters in fp16 so breaking down the threshold to `0` leads to a full model in `int8`. 